# Instrucciones.



Debería poder realizar todo el laboratorio en 3 horas.

Recomendamos fuertemente que siga trabajando en el laboratorio durante las horas asignadas, pues así puede consultar en vivo dudas que tenga sobre las partes finales del mismo.

Parte presencial.
* Complete la celda "A. Datos grupo".
* Descargue los archivos de instancias de ucursos.
* Complete las secciones B y C.
* Complete los ejercicios 1, 2 y 3 por el modulo de tareas de ucursos en la TAREA: Laboratorio 5 presencial
* Plazo de entrega: Miércoles 15 de julio a las 18:00

Parte no presencial.
* Complete todos los ejercicios (puede actualizar su parte presencial, pero debe INDICAR los cambios, agregándolos al final de la celda de Datos Grupo)
* Envie el archivo .ipynb por el modulo de tareas de ucursos en la TAREA: Laboratorio 5 completo
* Plazo de entrega: Domingo 19 de julio a las 23:59.

# A. Datos grupo

Escriba en esta celda el nombre de cada integrante de su grupo.

- Integrante 1: XXX

- Integrante 2: YYY

 # B. Preparación y Definición del problema


Usaremos los paquetes `JuMP, Gurobi, CSV, DataFrames`
Si le falta algún paquete instálelo (`using Pkg`; `Pkg.add("nombre_paquete")`) o actualícelo (`Pkg.update("nombre_paquete")`)
 
Al igual que en el laboratorio anterior, cree una constante `GUROBI_ENV` que guarde el ambiente gurobi que usaremos (de modo de evitar múltiples instancias). 

 

In [1]:
using JuMP, Gurobi, CSV, DataFrames
const GUROBI_ENV = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x00000000030dd9c0)

Considere el siguiente problema.
Se recibe un conjunto de $n$ trabajos $J$. Queremos procesar los trabajos de $J$ en algún orden, en una máquina de modo que cierta función se minimice. Para esto necesitamos un poco de notación

* Cada trabajo $s$ en $J$ (piense que $s$ es un nombre), tiene asignado un **tiempo de proceso** $p_s \in \mathbb{N}^+$, y un **deadline** $d_s$.

* Una **agenda** es un ordenamiento de los trabajos $\pi\colon J \to [n]$, donde $\pi(s)=i$ si el $i$-ésimo trabajo del órden es el trabajo $s$. Decimos que un trabajo $s$ se procesa antes que un trabajo $t$ si $\pi(s)<\pi(t)$.

* El **tiempo de partida** $S^{\pi}_s$ de $s$ de acuerdo a una agenda $\pi$ es la suma de los tiempos de proceso de todos los trabajos que se procesan antes que $j$. 
$$S^{\pi}_s=\sum_{t\in J\colon \pi(t)<\pi(s)}p_t$$

* El **tiempo de completación** $C^{\pi}_s$ de $s$ de acuerdo a una agenda $\pi$ es su tiempo de partida más su tiempo de proceso: 
$$C^{\pi}_s=S^{\pi}_s+p_s =\sum_{t\in J\colon \pi(t)\leq \pi(s)}p_t$$

* La **tardanza** de un trabajo $j$ de acuerdo a $\pi$ es $T^{\pi}_s= \max(C^{\pi}_s-d_s, 0)$

El objetivo del problema de las tardanzas es encontrar la agenda que minimiza la tardanza total de todos los trabajos, es decir queremos resolver:
$$\min_{\pi} \sum_{s\in J} T^{\pi}_s$$

Hoy compararemos distintas formulaciones para el problema de las tardanzas (que es NP-difícil) e intentaremos mejorar algunas de ellas.


# C. Formulaciones #

## C1. Formulación indexada por tiempo.###

Como la duración de los trabajos es entera positivo, podemos discretizar el tiempo. Sabemos que los tiempos de completación deben ser enteros entre 1 y $𝑃=\sum_{s\in J} p_s$, e introducir variables binarias  $C_{s,i}$, que valen 1 si y solo si el trabajo $s$  termina en el tiempo  $i$.

Con estas variables el problema se puede formular como

\begin{align*}
\min \sum_{s\in J}\sum_{i=d_s+1}^{P} (i-d_s)C_{s,i}\\
\sum_{i=p_s}^{P}C_{s,i} &= 1, \forall s \in J\\
\sum_{s\in J}\sum_{j=i}^{\min(P,i+p_s-1)}C_{s,j} &\leq 1, \forall i \in [P]\\
C_{s,i}&\in \{0,1\}
\end{align*}



## C2. Mediante variables de tiempo de completación. ##

Usaremos variables $C_s, T_s$  que representen el tiempo de completación y la tardanza del trabajo $s$ (en rigor, como los tiempos de proceso son números naturales, podríamos fijar $C_s$ a ser entero, esto no es necesario). Además, introducimos variables binarias $y_{s,t}$ para todo $s, t \in J, s\neq t$ donde $y_{s,t}=1$ si $s$ se procesa antes que $t$.

El problema se puede formular como

\begin{align*}
\min &\sum_{s\in J} T_s\\
T_s&\geq C_s - d_s, \forall s \in J \\
T_s&\geq 0, \forall s \in J\\
C_s&\geq p_s, \forall s \in J\\
C_s+p_t&\leq C_t + Py_{t,s}, \forall s,t \in J\\
y_{s,t}+y_{t,s}&=1, \forall s,t \in J, s\neq t\\
y_{s,t}&\in \{0,1\}
\end{align*}

Donde $P=\sum_{s\in J}p_s$. 

## C3. Formulación por ordenes totales. ##

La idea de esta formulación es parecida a la anterior pero nos permite eliminar el coeficiente $P$ que normalmente dificulta la ejecución de un PLE (formulaciones tipo "big-M"). Usamos variables $y_{s,t}$ binarias que valgan 1 cuando el trabajo $s$
aparece antes que el trabajo $t$ (o $s=t$). Notamos que las variables definen un orden total si y solo si
hay reflexividad, antisimetría y transitividad. Esto se puede imponer pidiendo que para $r, s, t$ se tenga:
$y_{s,s}=1$, $y_{s,t}+y_{t,s}=1$ si $s\neq t$; y que $y_{s,t}+y_{t,r}-1\leq y_{s,r}$
La última ecuación se puede hacer más simétrica notando que solo la necesitamos cuando $s,t,r$ son distintos. En ese caso, podemos reemplazar $y_{s,t}$ por antisimetría y tenemos $y_{s,t}+y_{t,r}+y_{r,s}\leq 2$

\begin{align*}
\min &\sum_{s\in J} T_s\\
T_s&\geq C_s - d_s \\
T_s&\geq 0\\
C_s&=\sum_{t\in J}y_{t,s}p_t\\
y_{s,s}&=1\\
y_{s,t}+y_{t,s}&=1, \text{si $s\neq t$}\\
y_{s,t}+y_{t,r}+y_{r,s}&\leq 2, \text{si $r, s, t$ distintos}\\
y_{s,t}&\in \{0,1\}
\end{align*}





## C4. Instancias ##

La siguiente función permite leer instancias del problema de la tardanza.
Modo de uso 
```julia
proceso, deadline = LeeArchivo(nombrearchivo)
```

In [2]:
function LeeArchivo(archivocsv)
    M= Matrix(CSV.read(archivocsv,  header=0))
    return M[1,:], M[2,:]
end

LeeArchivo (generic function with 1 method)

# D. Ejercicios #


### Ejercicio 1 ###

Implementar las 3 formulaciones en una función 

```Julia
function Tardanza(i, proceso, deadline)
return modelo
```
donde "i" es 1, 2, o 3
proceso, deadline son arreglos de tamaño $n$ que tienen en su $i$-ésima coordenada, el tiempo de proceso y el deadline del trabajo $i$.

**IMPORTANTE** 
(si no implementa las siguientes caracteristicas tendrá un descuento importante en la revisión)
* Agregue como "warm start" la solución donde los trabajos están ordenados de $1$ a $n$ (En todos los modelos hay una forma de hacer esto de manera rápida).
* Ponga como cota fija de tiempo de ejecución de solver: 30 segundos.

Puede probar sus metodos en la instancia **instancia0.csv** que se lee como se indica  en la parte C4. El valor objetivo de dicha instancia es 237.

In [349]:
function Tardanza(i, proceso, deadline)
    Tardanza = Model(optimizer_with_attributes(
                () -> Gurobi.Optimizer(GUROBI_ENV), "TimeLimit" => 30, "OutputFlag" => false))
    n = length(proceso)
    P = sum(proceso)
    if i==1
        @variable(Tardanza, C[1:n,1:P], Bin)
        @constraints(Tardanza, begin
        Unico[s=1:n], sum(C[s,i] for i = proceso[s]:P)==1     
        Termino[i=1:P], sum(C[s,j] for s=1:n, j=i:min(P,i+proceso[s]-1))<=1         
        end)
        @objective(Tardanza, Min, sum((i-deadline[s])*C[s,i] for s=1:n, i=deadline[s]+1:P))
        auxC=zeros(n,P)
        M=0
        for i in 1:n
                M+=proceso[i]
                auxC[i,M]=1
        end
        set_start_value.(C, auxC)
    else
        @variables(Tardanza, begin
            C[1:n]>=0
            T[1:n]>=0
            y[1:n, 1:n], Bin
        end)
        @objective(Tardanza, Min, sum(T) )
        @constraints(Tardanza, begin
        cota1[s=1:n], T[s]>=C[s]-deadline[s]
        anti[s=1:n, t=1:n; s!=t], y[s,t]+y[t,s]==1
        end)
        if (i==2)
           @constraints(Tardanza, begin
            cota2[s=1:n], C[s]>=proceso[s]
            conflicto[s=1:n, t=1:n; s!=t], C[s]+proceso[t] <= C[t]+P*y[t,s]  
            end) 
        elseif (i==3)
            @constraints(Tardanza, begin   
            completacion[s=1:n], C[s]==sum(y[t,s]*proceso[t] for t=1:n)
            refl[s=1:n], y[s,s]==1
            trans[s=1:n, t=1:n, r=1:n; s!=t!=r], y[s,t]+y[t,r]+y[r,s]<=2
            end)
            
        end
        auxC=zeros(n)
        auxT=zeros(n)
        auxy=zeros(n,n)
        for i in 1:n, j in 1:n
                if i<=j
                        auxy[i,j]=1
                end
        end
        M=0
        for i in 1:n
                M+=proceso[i]
                auxC[i]=M
                auxT[i]=max(auxC[i]-deadline[i],0)
        end
        set_start_value.(C, auxC)
        set_start_value.(T, auxC)
        set_start_value.(y, auxy)
    
    end
    return Tardanza
end

Tardanza (generic function with 1 method)

### Ejercicio 2 ###

Implemente una función

```julia 
function Resuelve(i, proceso, deadline)
return valor, orden, tiempo, gap
```

donde la entrada es igual que en la parte anterior. Esta funcion debe devolver

valor  = valor objetivo encontrado.
orden  = un arreglo con un orden de los trabajos en la solucion encontrada
        Ejemplo: [1 3 4 2] significa que la agenda pone a 1 en primer lugar, 3 en segundo, 4 en tercero y 2 en último.
        (debido al warm-start de la parte anterior, su programa debería siempre encontrar solución).
tiempo = tiempo de ejecución usado
gap    = gap de resolución en porcentaje (0% = optimo)


In [340]:
function Resuelve(i, proceso, deadline)
    modelo = Tardanza(i, proceso, deadline)
    tiempo = @elapsed optimize!(modelo)
    
    valor = objective_value(modelo)
    gap = MOI.get(modelo, MOI.RelativeGap())
    
    if i==1
        C = value.(modelo[:C])
        orden = [elem[1] for elem in sort(argmax(C,dims=2), dims=1, rev=false)][:,1]
    else
        y = value.(modelo[:y])
        orden = sortperm(sum(y, dims=2)[:,1], rev=true)
    end
    
    return valor, orden, tiempo, gap
end

Resuelve (generic function with 1 method)

In [342]:
proceso, deadline = LeeArchivo("instancia0.csv")

([5, 1, 5, 9, 9, 7, 10, 6, 9, 1], [1, 2, 1, 3, 3, 2, 1, 3, 2, 2])

In [459]:
valor, orden, tiempo, gap  = Resuelve(1, proceso, deadline)

(237.0, [10, 2, 3, 1, 8, 6, 4, 5, 9, 7], 0.015535632, 0.0)

In [460]:
valor, orden, tiempo, gap  = Resuelve(2, proceso, deadline)

(236.99999999998752, [2, 10, 3, 1, 8, 6, 9, 5, 4, 7], 1.09970163, 0.0)

In [461]:
valor, orden, tiempo, gap  = Resuelve(3, proceso, deadline)

(237.0, [10, 2, 3, 1, 8, 6, 4, 5, 9, 7], 0.00663147, 0.0)

## Symmetry Breaking ##

Las instancias provistas tienen mucha simetría. 

* Si dos trabajos $j$ y $k$ tienen igual tiempo de proceso entonces podemos "intercambiarlos" sin  altera la tardanza de ningún trabajo aparte de $j$ y $k$ (todos quedan con su mismo tiempo de completación). Solo cambia la tardanza de $j$ y $k$. De hecho, si llamamos $\pi$ a la agenda antigua y $\tau$ a la agenda intercambiada entonces los tiempos de completación de $j$ y de $k$ se intercambian. Luego, la diferencia entre la función objetivo nueva y la antigua es:

\begin{align*}
T^\tau_j + T^\tau_k - T^\pi_j - T^\pi_j &= (C^\tau_j-d_j) + (C^\tau_k-d_k) - (C^\pi_j-d_j) + (C^\pi_k-d_k)\\
&= (C^\pi_k-d_j)+(C^\pi_j-d_k) - (C^\pi_j-d_j) + (C^\pi_k-d_k)=0
\end{align*}

En otras palabras, si dos trabajos tienen el mismo tiempo de proceso, su orden relativo no importa y podemos elegir un orden a priori. Tomar decisiones a priori entre variables intercambiables es una manera de romper simetría que podría ayudar a que el árbol de BnB se simplifique (el solver no tiene que realizar el mismo trabajo en dos ramas que son iguales):


### Ejercicio 3. ###

Elija la formulación que quiera de las 3 y aplique symmetry breaking a los items con igual tiempo de proceso. Más precisamente: cuando escriba el modelo, particione los trabajos en grupos de trabajos con igual tiempo de proceso.
En cada grupo decida un orden fijo e imponga ese orden como restricciones adicionales.

En particular, cree funciones

```Julia
function TardanzaSB(proceso, deadline)
return modelo
```
```julia 
function ResuelveSB(proceso, deadline)
return valor, orden, tiempo, gap
```
que  hagan lo mismo que antes pero con esta formulación.

In [757]:
function TardanzaSB(proceso, deadline)
    Tardanza = Model(optimizer_with_attributes(
                () -> Gurobi.Optimizer(GUROBI_ENV), "TimeLimit" => 30, "OutputFlag" => false))
    n = length(proceso)
    P = sum(proceso)
    
    @variables(Tardanza, begin
        C[1:n]>=0
        T[1:n]>=0
        y[1:n, 1:n], Bin
    end)
    
    grupos = unique(proceso)
    m = length(grupos)
    for g in grupos
        trabajos = findall(x->x==g, proceso)
        n_trabajos = length(trabajos)
        if n_trabajos>1
            trabajos = hcat(trabajos, deadline[trabajos])
            prioridad = sortslices(trabajos, dims=1, by=x->x[2],rev=false)[:,1]
            for i=1:n_trabajos-1, j=i+1:n_trabajos
                fix(y[prioridad[i],prioridad[j]], 1; force=true)
            end
        end
    end
    
    @objective(Tardanza, Min, sum(T))
    @constraints(Tardanza, begin
    cota1[s=1:n], T[s]>=C[s]-deadline[s]
    anti[s=1:n, t=1:n; s!=t], y[s,t]+y[t,s]==1
    completacion[s=1:n], C[s]==sum(y[t,s]*proceso[t] for t=1:n)
    refl[s=1:n], y[s,s]==1
    trans[s=1:n, t=1:n, r=1:n; s!=t!=r], y[s,t]+y[t,r]+y[r,s]<=2
    end)
    
    return Tardanza
    
    
end

TardanzaSB (generic function with 1 method)

In [568]:
function ResuelveSB(proceso, deadline)
    modelo = TardanzaSB(proceso, deadline)
    tiempo = @elapsed optimize!(modelo)
    valor = objective_value(modelo)
    gap = MOI.get(modelo, MOI.RelativeGap())
    y = value.(modelo[:y])
    orden = sortperm(sum(y, dims=2)[:,1], rev=true)    
    
return valor, orden, tiempo, gap
end

ResuelveSB (generic function with 1 method)

In [569]:
ResuelveSB(proceso, deadline)

(237.0, [2, 10, 1, 3, 8, 6, 9, 4, 5, 7], 0.006930358, 0.0)

### Ejercicio 4 ###

En este problema deberá comparar las 3 formulaciones simples + la formulación con SB en las 10 instancias provistas 
instancia0.csv a instancia9.csv cada una con límite de tiempo de 30 segundos (y opciones estándar) 
    Más precisamente cree una función

```Julia
function Benchmark()
return df
```

donde df es un `dataframe` cuyas filas están indexadas por instancia, y las columnas son
    
| tiempo(1) | gap(1) | valor(1) | orden(1) | tiempo(2) | gap(2) | valor(2) | orden(2) | tiempo(3) | gap(3) | valor(3) | orden(3) | tiempo(SB) | gap(SB) | valor(SV) | orden(SB)

con
tiempo(j) = tiempo usado en la resolución de la instancia, usando formulacion 1
gap(j)    = gap de resolución en porcentaje (0% = optimo, infinito si no encontró solución factible)
valor(j)  = valor objetivo encontrado (infinito si no encontró solución)
orden(j)  = mejor orden encontrado 


In [755]:
function Benchmark()
    tiempo = zeros((10, 4))
    gap = zeros((10, 4))
    valor = zeros((10, 4))
    orden = []
    
    for i=1:10
        proceso, deadline = LeeArchivo("instancia$(i-1).csv")
        valor[i,1], orden1, tiempo[i,1], gap[i,1] = Resuelve(1, proceso, deadline)
        valor[i,2], orden2, tiempo[i,2], gap[i,2] = Resuelve(2, proceso, deadline)
        valor[i,3], orden3, tiempo[i,3], gap[i,3] = Resuelve(3, proceso, deadline)
        valor[i,4], orden4, tiempo[i,4], gap[i,4] = ResuelveSB(proceso, deadline)  
        push!(orden, [orden1, orden2, orden3, orden4])
        
    end
    df = DataFrame()
    for i=1:4
        col_tiempo = "tiempo$(i)"
        col_gap = "gap$(i)"
        col_valor = "valor$(i)"
        col_orden = "orden$(i)"
        df[!, Symbol(col_tiempo)] =  tiempo[:,i]
        df[!, Symbol(col_gap)] =  gap[:,i]
        df[!, Symbol(col_valor)] =  valor[:,i]
#         df[!, Symbol(col_orden)] =  [orden[j][i] for j=1:10]
    end
    
return df
end

Benchmark (generic function with 1 method)

In [759]:
df = Benchmark();

In [761]:
ENV["COLUMNS"]=120
df

,tiempo1,gap1,valor1,tiempo2,gap2,valor2,tiempo3,gap3,valor3,tiempo4,gap4,valor4
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0148399,0.0,237.0,1.10915,0.0,237.0,0.00742683,0.0,237.0,0.00602106,0.0,237.0
2,0.0428588,0.0,641.0,30.005,0.819701,650.0,0.189149,0.0,641.0,0.0631341,0.0,641.0
3,0.180591,0.0,1475.0,30.0101,0.901211,1569.0,16.7434,0.0,1475.0,0.622297,0.0,1475.0
4,0.18987,0.0,1710.0,30.0156,0.916667,1956.0,30.2843,0.0186916,1712.0,4.47746,0.0,1710.0
5,0.555295,0.0,3950.0,30.0489,0.933783,4591.0,30.491,0.00556962,3950.0,18.7776,0.0,3950.0
6,0.77143,0.0,5424.0,30.0297,0.947069,6688.0,31.0126,0.0115894,5436.0,30.8506,0.0075576,5425.0
7,0.857711,0.0,6262.0,30.1511,0.947667,7930.0,31.3439,1.0,9202.0,31.383,0.0103801,6262.0
8,1.76683,0.0,10607.0,30.0584,0.957102,12728.0,32.4811,1.0,14507.0,32.4617,0.00876201,10614.0
9,2.05128,0.0,11775.0,30.0656,0.957703,17590.0,34.3837,1.0,17701.0,34.3159,0.902586,17862.0
